In [1]:
import numpy as np
import os
import onnxruntime as rt
from onnxruntime import InferenceSession
from onnx.reference import ReferenceEvaluator

In [2]:
def load_embeddings(output_dir):
  input_matrix = np.load(os.path.join(output_dir, "embeddings.npy"))
  words = []
  with open(os.path.join(output_dir, "vocabulary.txt"), "r", encoding='utf-8') as f:
    for line in f.readlines():
      words.append(line.rstrip())
  return words, input_matrix

vocabulary, embeddings = load_embeddings('model/')

In [3]:
def get_hash(subword, bucket=2000000, nb_words=2000000):
  h = 2166136261
  for c in subword:
    c = ord(c) % 2**8
    h = (h ^ c) % 2**32
    h = (h * 16777619) % 2**32
  return h % bucket + nb_words
def get_subwords(word, vocabulary, minn=5, maxn=5):
  _word = "<" + word + ">"
  _subwords = []
  _subword_ids = []
  if word in vocabulary:
    _subwords.append(word)
    _subword_ids.append(vocabulary.index(word))
    if word == "</s>":
      return _subwords, np.array(_subword_ids)
  for ngram_start in range(0, len(_word)):
    for ngram_length in range(minn, maxn+1):
      if ngram_start+ngram_length <= len(_word):
        _candidate_subword = _word[ngram_start:ngram_start+ngram_length]
        if _candidate_subword not in _subwords:
          _subwords.append(_candidate_subword)
          _subword_ids.append(get_hash(_candidate_subword))
  return _subwords, np.array(_subword_ids)
def get_word_vector(word, vocabulary, embeddings):
  # subwords[1] contains the array of indices for the word and its subwords
  subword_ids = get_subwords(word, vocabulary)[1]
  
  # Check if the array of subword indices is empty
  if subword_ids.size == 0:
    # 💥 FIX: If no word/subword is found, return a 300-dimensional zero vector.
    # This ensures that all elements appended to the 'vectors' list have the same shape.
    embedding_dim = embeddings.shape[1] # This should be 300
    return np.zeros(embedding_dim)

  # Otherwise, compute the mean as before
  return np.mean([embeddings[s] for s in subword_ids], axis=0)
def tokenize(sentence):
  tokens = []
  word = ""
  for c in sentence:
    if c in [' ', '\n', '\r', '\t', '\v', '\f', '\0']:
      if word:
        tokens.append(word)
        word = ""
      if c == '\n':
        tokens.append("</s>")
    else:
      word += c
  if word:
    tokens.append(word)
  return tokens

def get_sentence_vector(line):
    tokens = tokenize(line)
    vectors = []
    for t in tokens:
        vec = get_word_vector(t, vocabulary, embeddings)
        norm = np.linalg.norm(vec)
        if norm > 0:
            vec /= norm
        vectors.append(vec)
    return np.mean(vectors, axis=0)

In [4]:
from pythainlp.corpus import thai_orst_words
import numpy as np
list_word=sorted(list(thai_orst_words()))
# list_word_use=[i for i in list_word if " " not in i]
words=list_word
words = np.array(list(words))

In [5]:
onnx_path = "nearest_neighbors.onnx" # Path where you saved the model

# 1. Load the model using InferenceSession
sess = rt.InferenceSession(onnx_path, providers=["CPUExecutionProvider"])

In [6]:
words_input = ['คดดี', 'สวัดี', 'vออกเลอร์', 'ปะเทศไทยย', 'อรอย']

In [7]:
word_input_vec = [get_sentence_vector(' '.join(list(word))) for word in words_input]
indices = sess.run(None, {"X": word_input_vec})[0] # n_neighbors is 5
suggestion = words[indices]

for w, s in zip(words_input, suggestion):
    print(f'{w} \n---> {s}')

คดดี 
---> ['คดี' 'คดีดำ' 'คนดี' 'คดีแดง' 'ดีดลูกคิด']
สวัดี 
---> ['วสวัดดี' 'สวัสดี' 'ดบัสวี' 'วิดัสดี' 'สรัสวดี']
vออกเลอร์ 
---> ['ออร์แกน' 'อาร์กอน' 'ออกเบอร์' 'เอทิลแอลกอฮอล์' 'คลอโรฟอร์ม']
ปะเทศไทยย 
---> ['กระเทียมโทน' 'กะเทย' 'ประโมทย์' 'คุยหประเทศ' 'ทรงกระเทียม']
อรอย 
---> ['รอย' 'อร่อย' 'ร่องรอย' 'เอร็ดอร่อย' 'ย้อนรอย']


In [8]:
list(zip(words_input, suggestion))

[('คดดี',
  array(['คดี', 'คดีดำ', 'คนดี', 'คดีแดง', 'ดีดลูกคิด'], dtype='<U50')),
 ('สวัดี',
  array(['วสวัดดี', 'สวัสดี', 'ดบัสวี', 'วิดัสดี', 'สรัสวดี'], dtype='<U50')),
 ('vออกเลอร์',
  array(['ออร์แกน', 'อาร์กอน', 'ออกเบอร์', 'เอทิลแอลกอฮอล์', 'คลอโรฟอร์ม'],
        dtype='<U50')),
 ('ปะเทศไทยย',
  array(['กระเทียมโทน', 'กะเทย', 'ประโมทย์', 'คุยหประเทศ', 'ทรงกระเทียม'],
        dtype='<U50')),
 ('อรอย',
  array(['รอย', 'อร่อย', 'ร่องรอย', 'เอร็ดอร่อย', 'ย้อนรอย'], dtype='<U50'))]

In [9]:
suggestion

array([['คดี', 'คดีดำ', 'คนดี', 'คดีแดง', 'ดีดลูกคิด'],
       ['วสวัดดี', 'สวัสดี', 'ดบัสวี', 'วิดัสดี', 'สรัสวดี'],
       ['ออร์แกน', 'อาร์กอน', 'ออกเบอร์', 'เอทิลแอลกอฮอล์', 'คลอโรฟอร์ม'],
       ['กระเทียมโทน', 'กะเทย', 'ประโมทย์', 'คุยหประเทศ', 'ทรงกระเทียม'],
       ['รอย', 'อร่อย', 'ร่องรอย', 'เอร็ดอร่อย', 'ย้อนรอย']], dtype='<U50')